In [1]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
api_key = "0140412acf0e4f69a00b0c63c454a2e8"
azure_endpoint = "https://thangtd.openai.azure.com/"
api_version = "2023-03-15-preview"
llm = AzureOpenAI(
    model="GPT35",
    deployment_name="gpt",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="ADA002",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [3]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [7]:
from llama_index.core.evaluation import DatasetGenerator, QueryResponseDataset
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.readers.file import FlatReader, HTMLTagReader
from llama_index.core.node_parser import HTMLNodeParser, SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import set_global_handler
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SemanticSplitterNodeParser

from pathlib import Path

reader = SimpleDirectoryReader("test_data/")

docs = reader.load_data()

In [8]:
print(len(docs))

1


In [10]:
docs

[Document(id_='e521ded0-aeb0-460f-9f18-2a9105f034bb', embedding=None, metadata={'file_path': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_name': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_type': 'text/html', 'file_size': 2626, 'creation_date': '2024-03-06', 'last_modified_date': '2024-03-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='<div>\n  <h1 class="c-ttl-article__txt">結婚12年目「倖田來未」が明かす“夫と息子”との私生活\u3000「親子でプロレスにハマってます」</h1>\n  <p>\n    今年でデビュー23年目を迎える歌手の倖田來未（40）。「前編」では、下積み時代から芸能界における浮き沈みを経験しながら、「エロかっこいい」と称される人気歌手に上り詰めるまでの半生を聞いた。そんな彼女は1月18日にMusic ＆\n    Live Package「WINGS」をリリースした。収録曲の作曲には夫も参加しているが、今回の「後編」では主に夫や息子など家族がアーティスト・倖田來未にどのような影響を与えているかをインタビューした。\n  </p>\n  <p>\n    倖田來未が1月18